In [1]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

C:\Users\Nolo42\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
#Import Data
ldf1 = pd.read_csv('data/loan1.csv')
ldf2 = pd.read_csv('data/loan2.csv')
ldf3 = pd.read_csv('data/loan3.csv')
ldf4 = pd.read_csv('data/loan4.csv')
df = ldf1
df = df.append(ldf2)
df = df.append(ldf3)
df = df.append(ldf4)

C:\Users\Nolo42\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Nolo42\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Calculated column simplifying loan status
df["badloan"]=df.loan_status
df=df.dropna(subset=['badloan'])  

In [4]:
#Classify loans as being in good or bad standing
def lstat(status):
    if status == "Charged Off" or status == "Default" or status == "Does not meet the credit policy.  Status:Charged Off":
        return "Bad"
    else:
        return "Good"
df.badloan=df.badloan.apply(lstat)

In [5]:
#Fix data type
df.fico_range_low.convert_objects(convert_numeric=True)
df.fico_range_high.convert_objects(convert_numeric=True)

C:\Users\Nolo42\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app
C:\Users\Nolo42\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


0         739
1         744
2         739
3         694
4         699
5         734
6         694
7         664
8         679
9         729
10        699
11        679
12        714
13        709
14        724
15        669
16        674
17        764
18        689
19        759
20        729
21        709
22        694
23        714
24        694
25        734
26        699
27        679
28        684
29        699
         ... 
290561    709
290562    674
290563    674
290564    694
290565    744
290566    724
290567    679
290568    709
290569    674
290570    679
290571    729
290572    704
290573    759
290574    689
290575    684
290576    714
290577    829
290578    669
290579    749
290580    674
290581    674
290582    684
290583    674
290584    689
290585    684
290586    674
290587    664
290588    684
290589    699
290590    679
Name: fico_range_high, dtype: float64

In [6]:
#Create working dataframe
fdf=df[['loan_amnt','home_ownership','annual_inc', 'fico_range_high','badloan']]

In [7]:
#Remove instances with missing data
fdf=fdf.dropna(axis=0)

In [8]:
#Simplify living situation
def ownsh(status):
    if status == "RENT":
        return 1
    elif status == "OWN":
        return 2
    elif status == "MORTGAGE":
        return 3
    else:
        return 4
fdf['home_ownership']=fdf['home_ownership'].apply(ownsh)

In [9]:
#Further simplify loan status
def blon(status):
    if status == "Bad":
        return 1
    elif status == "Good":
        return 2
fdf['badloan']=fdf['badloan'].apply(blon)

In [10]:
#Clear some missing data
fdf=fdf.dropna(axis=0)

In [11]:
#Separate label and feature data
labeldf=fdf.badloan
del fdf['badloan']
features_train=fdf.iloc[0:40000].values
features_test=fdf.iloc[40001:60000].values
labels_train=labeldf.iloc[0:40000].values
labels_test=labeldf.iloc[40001:60000].values

In [14]:
#Support vector machine
clf = SVC()
clf.fit(features_train, labels_train) 
pred1 = clf.predict(features_test)
print(accuracy_score(pred1, labels_test))
print(f1_score(labels_test, pred1, average='weighted'))

0.888844442222
0.0330578512397


C:\Users\Nolo42\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:931: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)


In [13]:
#Support vector machine2
clf = SVC(class_weight='auto')
clf.fit(features_train, labels_train) 
pred2 = clf.predict(features_test)
print(accuracy_score(pred2, labels_test))
print(f1_score(labels_test, pred2, average='weighted'))

0.86389319466
0.0697197539303


C:\Users\Nolo42\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:931: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)


In [15]:
#Decision tree
clft = tree.DecisionTreeClassifier(min_samples_split=2)
clft.fit(features_train, labels_train)
pred4 = clft.predict(features_test)
print(accuracy_score(pred4, labels_test))
print(f1_score(labels_test, pred4, average='weighted'))

0.740987049352
0.151654110711


C:\Users\Nolo42\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:931: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)


In [16]:
#Random forest
clfrf = RandomForestClassifier(n_estimators=10)
clfrf = clfrf.fit(features_train, labels_train)
pred5 = clfrf.predict(features_test)
print(accuracy_score(pred5, labels_test))
print(f1_score(labels_test, pred5, average='weighted'))

C:\Users\Nolo42\Anaconda3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Users\Nolo42\Anaconda3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Users\Nolo42\Anaconda3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Users\Nolo42\Anaconda3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Users\Nolo42\Anaconda3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, var

0.813990699535
0.114707282247
